# Network Data Activities -- APIs!!

Remember, web calls are made using the requests package:

https://www.w3schools.com/python/ref_requests_get.asp

This is are, in fact, just webcalls.  We can use it to access *any website*.

In [ ]:
import requests

response = requests.get("http://sci.pitt.edu/this-api-doesnt-exist")

print(response.status_code)

In [ ]:
print(response.content)

In [ ]:
response = requests.get("http://sci.pitt.edu/")
print(response.status_code)

In [ ]:
print(response.apparent_encoding)

In [ ]:
print(response.content)

Calling to get a website is not interesting, because it's really hard for us to make use of that data...

But other web calls are not just for getting websites, but for getting data from websites.  Specifically, JSON structured data is common.

For this activity, we're going to work with an API, JSON, along with some date/time manipulations to do some Space Stuff.  Specifcally, were going to use an API to learn more about the International Space Station.

API refernece: http://api.open-notify.org

To begin, let's look at the documentation and write a request to get the list of people in space *right now*.

In [ ]:
response = requests.get("http://api.open-notify.org/astros.json")
print(response.status_code)
print(response.json())

Now, let's make this list pretty by breaking down the JSON and printing a clean list.

In [ ]:
response.json()

In [ ]:
astros = response.json()['people']

for astro in astros :
    print (astro['name'] + ", " + astro['craft'])

Ok, now let's make this a little more *kinetic*... can we use the iss-pass API to learn when the next time the space station will pass over Pittsburgh??

Hint, the University of Pittsburgh's long and lat are 79.9959 W and 40.4406 N

Another hint, we need to use the params feature of requests!

API to use: http://open-notify.org/Open-Notify-API/ISS-Pass-Times/

In [ ]:
response = requests.get("http://api.open-notify.org/iss-pass.json", params='lon=-79.9959&lat=40.4406')

print(response.json())

In [ ]:
response.json()['response']

Ok, for the challenging part of this Activity, let's use the datatime package to convert these UNIX time to useful time and print out the pass times.

Reference for the datatime package:
https://docs.python.org/3/library/datetime.html

In [ ]:
from datetime import datetime
for passby in response.json()['response'] :
    riseTime = datetime.utcfromtimestamp(passby['risetime'])
    print(str(riseTime) + ' ' + str(passby['duration'] // 60) + ':' + str(passby['duration'] % 60).zfill(2))

But wait, this is UTC time,  right now in Daylight time on the East Coast, we are 4 hours *behind* UTC.  Can we use the datetime functions to subtract an hour?

What about this timedelta object?
https://docs.python.org/3/library/datetime.html#timedelta-objects

In [ ]:
from datetime import datetime, timedelta
for passby in response.json()['response'] :
    riseTime = datetime.utcfromtimestamp(passby['risetime'])
    riseTime = riseTime - timedelta(hours=4)
    print(str(riseTime) + ' ' + str(passby['duration'] // 60) + ':' + str(passby['duration'] % 60).zfill(2))

Bonus activity!!!  Can we print the time to look up, and hor how long in a more pretty way?  What if it's overhead **right now**??

For instnace, something like:

Look up in: 9 hours 22 minutes 5 seconds for 9 minutes 1 seconds

Look up in: 10 hours 57 minutes 32 seconds for 10 minutes 54 seconds

Look up in: 12 hours 35 minutes 19 seconds for 9 minutes 53 seconds

Look up in: 14 hours 13 minutes 29 seconds for 9 minutes 15 seconds

Look up in: 15 hours 50 minutes 41 seconds for 10 minutes 19 seconds

In [ ]:
from datetime import datetime

currentTime = datetime.now()

for passby in response.json()['response'] :
    riseTime = datetime.fromtimestamp(passby['risetime'])
    
    timeUntil = riseTime - currentTime
    
    if(riseTime < currentTime) :
        
        timeLeft = -((60*60*24 - timeUntil.seconds)) + passby['duration']
        
        if(timeLeft > 0) :
            print('Look up now! for the next ' + str(timeLeft // 60) +
                  ' minutes ' + str(timeLeft % 60) + ' seconds')

    else :
        
    
        print('Look up in: ' + str(timeUntil.seconds // (60 * 60)) + ' hours ' +
              str(((timeUntil.seconds // (60 * 60)) * -60) + timeUntil.seconds // 60) + ' minutes ' + str(timeUntil.seconds % 60) + ' seconds for ' +
              str(passby['duration'] // 60) + ' minutes ' + str(passby['duration'] % 60) + ' seconds')

Let's check our work...

https://spotthestation.nasa.gov/tracking_map.cfm